In [58]:
import requests
import pandas as pd
import numpy as np
import json
import time

strava = pd.read_excel('Strava Running Data.xlsx')
strava.head()

,Sr. no.,start_date_local,type,distance,moving_time,elapsed_time,total_elevation_gain,start_latlng,end_latlng,sport_type,start_date,timezone,achievement_count,kudos_count,comment_count,athlete_count,photo_count,average_speed,max_speed
0,1,2023-12-09T09:09:19Z,Run,10879.7,4023,4617,91.4,"[40.70327935740352, -73.99619171395898]","[40.69340907968581, -73.97922154515982]",Run,2023-12-09T14:09:19Z,(GMT-05:00) America/New_York,4,4,0,3,0,2.704,6.228
1,2,2023-12-07T17:31:50Z,Run,1304.4,722,62993,0.0,"[40.7220459356904, -74.03641730546951]","[40.719722136855125, -74.03257705271244]",Run,2023-12-07T22:31:50Z,(GMT-05:00) America/New_York,0,0,0,1,0,1.807,4.044
2,3,2023-12-03T09:18:13Z,Run,17503.0,7370,7462,68.4,"[40.74991073459387, -73.98751585744321]","[40.73538766242564, -73.9793517999351]",Run,2023-12-03T14:18:13Z,(GMT-05:00) America/New_York,2,5,0,4,0,2.375,6.778
3,4,2023-12-02T09:41:14Z,Run,3457.8,1791,2170,3.9,"[40.72203076444566, -74.03641068376601]","[40.71398631669581, -74.03902692720294]",Run,2023-12-02T14:41:14Z,(GMT-05:00) America/New_York,0,2,0,3,0,1.931,4.522
4,5,2023-12-01T17:06:05Z,Run,10108.2,4128,4221,6.3,"[40.72190687991679, -74.03624514117837]","[40.71669920347631, -74.03264125809073]",Run,2023-12-01T22:06:05Z,(GMT-05:00) America/New_York,4,5,0,1,0,2.449,5.462


On supprime les lignes sans latitude ou longitude

In [59]:
for l in strava['end_latlng']:
    if len(l.split(','))==1:
        print(l)

[]
[]
[]
[]
[]
[]
[]
[]
[]


In [60]:
def extract_lat_lng(s):
    s = s.replace('[', '').replace(']', '')
    parts = s.split(',')
    if len(parts) != 2:
        return pd.Series({'lat': np.nan, 'lng': np.nan})
    try:
        lat = float(parts[0].strip())
        lng = float(parts[1].strip())
    except ValueError:
        lat, lng = np.nan, np.nan
    return pd.Series({'lat': lat, 'lng': lng})

strava[['lat', 'lng']] = strava['start_latlng'].apply(extract_lat_lng)

strava.head()

,Sr. no.,start_date_local,type,distance,moving_time,elapsed_time,total_elevation_gain,start_latlng,end_latlng,sport_type,...,timezone,achievement_count,kudos_count,comment_count,athlete_count,photo_count,average_speed,max_speed,lat,lng
0,1,2023-12-09T09:09:19Z,Run,10879.7,4023,4617,91.4,"[40.70327935740352, -73.99619171395898]","[40.69340907968581, -73.97922154515982]",Run,...,(GMT-05:00) America/New_York,4,4,0,3,0,2.704,6.228,40.703279,-73.996192
1,2,2023-12-07T17:31:50Z,Run,1304.4,722,62993,0.0,"[40.7220459356904, -74.03641730546951]","[40.719722136855125, -74.03257705271244]",Run,...,(GMT-05:00) America/New_York,0,0,0,1,0,1.807,4.044,40.722046,-74.036417
2,3,2023-12-03T09:18:13Z,Run,17503.0,7370,7462,68.4,"[40.74991073459387, -73.98751585744321]","[40.73538766242564, -73.9793517999351]",Run,...,(GMT-05:00) America/New_York,2,5,0,4,0,2.375,6.778,40.749911,-73.987516
3,4,2023-12-02T09:41:14Z,Run,3457.8,1791,2170,3.9,"[40.72203076444566, -74.03641068376601]","[40.71398631669581, -74.03902692720294]",Run,...,(GMT-05:00) America/New_York,0,2,0,3,0,1.931,4.522,40.722031,-74.036411
4,5,2023-12-01T17:06:05Z,Run,10108.2,4128,4221,6.3,"[40.72190687991679, -74.03624514117837]","[40.71669920347631, -74.03264125809073]",Run,...,(GMT-05:00) America/New_York,4,5,0,1,0,2.449,5.462,40.721907,-74.036245


In [61]:
strava.dropna(inplace=True)

In [62]:
try:
    strava['start_date_local'] = pd.to_datetime(strava['start_date_local'], errors='coerce').dt.strftime('%Y-%m-%d')
except Exception as e:
    print(f"Erreur lors de la conversion des dates : {e}")

url = "https://archive-api.open-meteo.com/v1/era5"
weather_data = []

for _, row in strava.iterrows():
    latitude = row['lat']
    longitude = row['lng']
    date = row['start_date_local']
    index = row["Sr. no."]

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "timezone": "America/New_York",
        "start_date": date,
        "end_date": date,
        "daily": ["rain_sum", "temperature_2m_max", "temperature_2m_min", "snowfall_sum",  "precipitation_sum", "sunshine_duration", "shortwave_radiation_sum","wind_speed_10m_max"]
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()

        if "daily" in data:
            daily_data = data["daily"]

            rain_sum = daily_data["rain_sum"][0] if "rain_sum" in daily_data else None
            max_temp = daily_data["temperature_2m_min"][0] if "temperature_2m_min" in daily_data else None
            min_temp = daily_data["temperature_2m_max"][0] if "temperature_2m_max" in daily_data else None
            snowfall_sum = daily_data["snowfall_sum"][0] if "snowfall_sum" in daily_data else None
            wind_speed = daily_data["wind_speed_10m_max"][0] if "wind_speed_10m_max" in daily_data else None
            sunshine_duration = daily_data["sunshine_duration"][0] if "sunshine_duration" in daily_data else None
            total_precipitation = daily_data["precipitation_sum"][0] if "precipitation_sum" in daily_data else None
            solar_radiation = daily_data["shortwave_radiation_sum"][0] if "shortwave_radiation_sum" in daily_data else None


            # Ajouter les données collectées dans la liste weather_data
            weather_data.append({
                "Index": index,
                "Date": date,
                "MaxTemperature": max_temp,
                "MinTemperature": min_temp,
                "TotalPrecipitation": total_precipitation,
                "MaxWindSpeed": wind_speed,
                "SolarRadiation": solar_radiation,
                "SunshineDuration": sunshine_duration,
                "RainSum":rain_sum,
                "SnowFall":snowfall_sum,
                
                
            })

    except Exception as e:
        print(f"Erreur pour l'index {index}: {e}")


weather_df = pd.DataFrame(weather_data)

merged_df = pd.merge(strava, weather_df, left_on=["Sr. no.", "start_date_local"], right_on=["Index", "Date"], how="left")

# Optionnel : supprimer les colonnes redondantes 'Index' et 'Date'
merged_df = merged_df.drop(columns=["Index", "Date"])

# Vous pouvez maintenant utiliser merged_df qui contient les données météo intégrées


In [63]:
merged_df.isna().sum()

Sr. no.                 0
start_date_local        0
type                    0
distance                0
moving_time             0
elapsed_time            0
total_elevation_gain    0
start_latlng            0
end_latlng              0
sport_type              0
start_date              0
timezone                0
achievement_count       0
kudos_count             0
comment_count           0
athlete_count           0
photo_count             0
average_speed           0
max_speed               0
lat                     0
lng                     0
MaxTemperature          0
MinTemperature          0
TotalPrecipitation      0
MaxWindSpeed            0
SolarRadiation          0
SunshineDuration        0
RainSum                 0
SnowFall                0
dtype: int64

In [64]:
merged_df.dropna(inplace=True)

In [65]:
merged_df.to_csv('Combined_data.csv')